LANGSMITH_API_KEY : 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

TOGETHER_API_KEY : 2dde51635fbb84097198ddb134f5b04d580a467bc973e56ee9b340759336cb97



In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true" # 启用 LangChain V2 跟踪，可以在 https://smith.langchain.com/中查看跟踪数据
os.environ["TOGETHER_API_KEY"] = '2dde51635fbb84097198ddb134f5b04d580a467bc973e56ee9b340759336cb97' # 模型api key
os.environ["LANGSMITH_API_KEY"] = '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' # langsmith api key
# os.environ["OPENAI_API_KEY"] = ''


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

# -----------

In [19]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="hello world!"),
]

result = model.invoke(messages)
result

AIMessage(content=' "hello world!" in Chinese is "你好，世界！" (nǐ hǎo, shìjiè!)\n\nThe Chinese language has many variations and dialects, but the most commonly used written form is Simplified Chinese, which is based on the Mandarin dialect. The characters I provided are Simplified Chinese characters.\n\nIn Traditional Chinese, which is used in Taiwan and Hong Kong, the same phrase would be written as "你好，世界！" (nǐ hǎo, shìjiè!) with the same pronunciation.\n\nI hope this helps! Let me know if you have any other questions.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 140, 'prompt_tokens': 21, 'total_tokens': 161, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'id': 'o4xfthb-62bZhn-96697392cc359e6b', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--416c9ce7-22f3-4288-8a81-240b3eacf620-0', usage_metadata={'input_tokens

# 结果解析方式
- StrOutputParser()：提取字符串
- JsonOutputParser()：解析 JSON 输出
- .content：获取原始输出内容


In [4]:
# 方式1 解析器
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [5]:
parser.invoke(result)

' "Hello World!" in Chinese is "你好，世界!" which is pronounced as "Nǐ hǎo, shìjiè!" in Mandarin. Here\'s the breakdown of the sentence:\n\n* "Nǐ" (你) means "you"\n* "Hǎo" (好) means "good" or "hello" in this context\n* "Shìjiè" (世界) means "world"\n\nSo, when you put it together, you get "Hello, World!" in Chinese.'

In [6]:
# 方式2 content属性获取结果str
result.content

' "Hello World!" in Chinese is "你好，世界!" which is pronounced as "Nǐ hǎo, shìjiè!" in Mandarin. Here\'s the breakdown of the sentence:\n\n* "Nǐ" (你) means "you"\n* "Hǎo" (好) means "good" or "hello" in this context\n* "Shìjiè" (世界) means "world"\n\nSo, when you put it together, you get "Hello, World!" in Chinese.'

In [ ]:
# 管道链接，将模型和解析器链接起来，逐个调用
chain = model | parser
print(chain.invoke(messages))

 "hello world!" in Chinese is "你好，世界！" (nǐ hǎo, shìjiè!)

Please note that the tone marks are not displayed correctly in some text editors or devices. The pinyin for the Chinese characters is provided to help with the pronunciation.


In [ ]:

# StrOutputParser()：提取字符串
# JsonOutputParser()：解析 JSON 输出
# PydanticOutputParser()：转换为结构化数据对象
# RetryWithErrorOutputParser()：支持错误重试解析

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser

stroutparser, jsonoutparser = StrOutputParser(), JsonOutputParser()

print(result.content, end='\n')
print(stroutparser.invoke(result), end='\n')
print(jsonoutparser.invoke(result), end='\n')


# PromptTemplates 示例

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "将以下内容翻译为{language}，对话采用中文:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='将以下内容翻译为{language}，对话采用中文:'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [16]:
prompt_result = prompt_template.format_prompt(text="你好，世界！", language="Japanese")
prompt_result

ChatPromptValue(messages=[SystemMessage(content='将以下内容翻译为Japanese，对话采用中文:', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好，世界！', additional_kwargs={}, response_metadata={})])

In [17]:
prompt_result.to_messages()

[SystemMessage(content='将以下内容翻译为Japanese，对话采用中文:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好，世界！', additional_kwargs={}, response_metadata={})]

In [18]:
chain = prompt_template | model | parser
print(chain.invoke({'text': "你好，世界！", 'language' : "Japanese"}))

 こんにちは、sekai!

Note: In Japanese, "sekai" means "world", but it is a common and grammatically correct way to say "world" in this context in Japanese. The word "sekai" itself is often used in Japanese greetings, songs, and sayings, similar to how "world" is used in the English greeting "Hello, world!".


# 聊天机器人

In [21]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")]).content

" Hello Bob! It's nice to meet you. How can I assist you today? If you have any questions or need help with something, feel free to ask. I'm here to help!"

In [22]:
model.invoke([HumanMessage(content="What's my name?")]).content

" I'm unable to determine your name as I don't have any personal data about you, unless you're willing to share it with me. However, I'm here to provide information and answer questions to the best of my ability."

# Test

In [2]:
l = [
  "\r\n",
  "\r\n      \r\n          ",
  "\r\n      \r\n ",
  "\r\n      \r\n      发布于 2025年07月29日 08:42   更新于 2025年07月29日 17:06\r\n      \r\n      ",
  "\r\n      \r\n      \r\n      ",
  "\r\n      \r\n      ",
  "\r\n     "
]

In [3]:
pub_info = next((t.strip() for t in l if "发布于" in t), None)
pub_info

'发布于 2025年07月29日 08:42\xa0\xa0\xa0更新于 2025年07月29日 17:06'

In [11]:
import re
match = re.search(r"\d{4}年\d{2}月\d{2}日 \d{2}:\d{2}", pub_info or "")
match.group()

'2025年07月29日 08:42'

In [12]:
import re

# 提取发布时间、更新时间（如果有）
pub_time = None
update_time = None

match = re.search(r"发布于 (\d{4}年\d{2}月\d{2}日 \d{2}:\d{2})", pub_info or "")
if match:
    pub_time = match.group()

match = re.search(r"更新于 (\d{4}年\d{2}月\d{2}日 \d{2}:\d{2})", pub_info or "")
if match:
    update_time = match.group()

In [13]:
pub_time,update_time

('发布于 2025年07月29日 08:42', '更新于 2025年07月29日 17:06')

# Test

In [1]:
import re

In [30]:
s = '''
<h4><a href="https://china.caixin.com/2025-07-31/102347095.html">海南落马高官倪强被罢免全国人大代表</a>
'''
# s = s.strip()
pat = r'/\d{4}-\d{2}-\d{2}/\d+\.html'

In [32]:
res = re.search(pat, s)
print(res.group())

/2025-07-31/102347095.html


In [ ]:
import pandas as pd
pd.DataFrame([{'a': 1, 'b': 2}, {'a': 3, 'b': 4}])

In [2]:
import urllib.request
import os
from urllib.parse import urlparse

In [3]:
url = "https://ts1.tc.mm.bing.net/th/id/R-C.d2b3a4779fd2b9af70103d485bc8b664?rik=Xm7zutXMpsp91Q&riu=http%3a%2f%2fup.deskcity.org%2fpic_source%2fd2%2fb3%2fa4%2fd2b3a4779fd2b9af70103d485bc8b664.jpg&ehk=%2fh%2fipXq8Ihn81SbQdkphnzweLFLUGfD1%2fXncDcbLgRE%3d&risl=&pid=ImgRaw&r=0"

In [20]:
os.makedirs('tmp',exist_ok=True)

In [21]:
# 查看当前路径
print(os.getcwd())

D:\Codes\py\Agent


In [22]:
parsed_url = urlparse(url)
img_base_name = "img"
num = 0
filename = f"{img_base_name}_{num}.jpg"
num+=1

In [23]:
file_path = os.path.join('tmp', filename)
file_path

'tmp\\img_0.jpg'

In [24]:
req = urllib.request.Request(url)
with urllib.request.urlopen(req, timeout=30) as response:
    with open(file_path, 'wb') as f:
        f.write(response.read())

print(f"图片已保存到: {file_path}")

图片已保存到: tmp\img_0.jpg


In [10]:
from typing import TypedDict, List, Dict
import re

async def func(list) :
    text_cap = []
    # 优化后的正则表达式
    pattern = r'([，。；？！])'  # 只关注指定标点

    for item in list:
        # 分阶段处理
        raw_cap = item['cap']

        # 第一步：按标点分割
        segments = re.split(pattern, raw_cap)

        # 第二步：重组并过滤
        sentences = []
        current_sentence = ""
        for seg in segments:
            if seg.strip():
                # 判断是否是标点符号
                if seg in ('，', '。', '；', '？', '！'):
                    current_sentence += seg
                    # 处理末尾符号
                    if seg in ('？', '！'):
                        sentences.append(current_sentence)
                    else:
                        sentences.append(current_sentence[:-1])  # 去掉末尾标点
                    current_sentence = ""
                else:
                    current_sentence = seg

        # 处理最后残余内容
        if current_sentence:
            sentences.append(current_sentence)

        text_cap.extend(sentences)

    return {"text_cap": text_cap}

In [13]:
list = [
    {
      "story_name": "开场引入",
      "desc": "简约的黑色线条勾勒出一个地球仪，旁边站着一位拿着放大镜的商务人士，背景是淡蓝色。",
      "cap": "你知道吗？注册新加坡公司，可能比你想象中简单得多！而且，它的优势远远超乎你的预期。今天，我就带你一探究竟，看看为什么越来越多的企业家选择在新加坡注册公司。"
    },
    {
      "story_name": "金融中心介绍",
      "desc": "新加坡城市天际线的剪影，金融区的高楼大厦，中间有一个金色的货币符号。",
      "cap": "首先，新加坡作为亚洲金融中心，拥有发达的金融市场和多元化的金融服务。无论是想拓展国际市场，还是规划税务，新加坡都是一个理想的选择。"
    },
    {
      "story_name": "低税率政策",
      "desc": "一个简单的税务表格，税率数字被绿色圆圈标记，旁边有一个微笑的表情符号。",
      "cap": "更重要的是，新加坡实行低税率政策，企业所得税率相对较低，尤其是新小型企业，还能享受税收减免的优惠。"
    },
    {
      "story_name": "双重征税协定",
      "desc": "两只手握手，背景是两个国家的国旗，中间有一个绿色的对勾符号。",
      "cap": "再加上新加坡与许多国家签订了双重征税协定，避免了双重征税的烦恼，真是省心又省钱！"
    }
]
text_cap = []
# 优化后的正则表达式
pattern = r'([，。；？！])'  # 只关注指定标点

for item in list:
    # 分阶段处理
    raw_cap = item['cap']

    # 第一步：按标点分割
    segments = re.split(pattern, raw_cap)

    # 第二步：重组并过滤
    sentences = []
    current_sentence = ""
    for seg in segments:
        if seg.strip():
            # 判断是否是标点符号
            if seg in ('，', '。', '；', '？', '！'):
                current_sentence += seg
                # 处理末尾符号
                if seg in ('？', '！'):
                    sentences.append(current_sentence)
                else:
                    sentences.append(current_sentence[:-1])  # 去掉末尾标点
                current_sentence = ""
            else:
                current_sentence = seg

    # 处理最后残余内容
    if current_sentence:
        sentences.append(current_sentence)

    text_cap.extend(sentences)

In [14]:
text_cap

['你知道吗？',
 '注册新加坡公司',
 '可能比你想象中简单得多！',
 '而且',
 '它的优势远远超乎你的预期',
 '今天',
 '我就带你一探究竟',
 '看看为什么越来越多的企业家选择在新加坡注册公司',
 '首先',
 '新加坡作为亚洲金融中心',
 '拥有发达的金融市场和多元化的金融服务',
 '无论是想拓展国际市场',
 '还是规划税务',
 '新加坡都是一个理想的选择',
 '更重要的是',
 '新加坡实行低税率政策',
 '企业所得税率相对较低',
 '尤其是新小型企业',
 '还能享受税收减免的优惠',
 '再加上新加坡与许多国家签订了双重征税协定',
 '避免了双重征税的烦恼',
 '真是省心又省钱！']

In [17]:
S = "你知道吗？注册新加坡公司，可能比你想象中简单得多！而且，它的优势远远超乎你的预期。今天，我就带你一探究竟，看看为什么越来越多的企业家选择在新加坡注册公司。"
s = "注册新加坡公司"
s in S

True

In [18]:
input_list =[
  {
      "story_name": "开场引入",
      "desc": "简约的黑色线条勾勒出一个地球仪，旁边站着一位拿着放大镜的商务人士，背景是淡蓝色。",
      "cap": "你知道吗？注册新加坡公司，可能比你想象中简单得多！而且，它的优势远远超乎你的预期。今天，我就带你一探究竟，看看为什么越来越多的企业家选择在新加坡注册公司。"
    },
    {
      "story_name": "金融中心介绍",
      "desc": "新加坡城市天际线的剪影，金融区的高楼大厦，中间有一个金色的货币符号。",
      "cap": "首先，新加坡作为亚洲金融中心，拥有发达的金融市场和多元化的金融服务。无论是想拓展国际市场，还是规划税务，新加坡都是一个理想的选择。"
    },
    {
      "story_name": "低税率政策",
      "desc": "一个简单的税务表格，税率数字被绿色圆圈标记，旁边有一个微笑的表情符号。",
      "cap": "更重要的是，新加坡实行低税率政策，企业所得税率相对较低，尤其是新小型企业，还能享受税收减免的优惠。"
    },
    {
      "story_name": "双重征税协定",
      "desc": "两只手握手，背景是两个国家的国旗，中间有一个绿色的对勾符号。",
      "cap": "再加上新加坡与许多国家签订了双重征税协定，避免了双重征税的烦恼，真是省心又省钱！"
    },
    {
      "story_name": "注册条件",
      "desc": "一个简单的表格，列出年龄、国籍、董事、地址和秘书等条件，旁边有一个绿色的勾。",
      "cap": "那么，注册新加坡公司到底需要哪些条件呢？其实并不复杂。只要年满18周岁，不限国籍，都可以在新加坡注册公司。不过，董事中至少有一名必须是新加坡本地居民。此外，公司还需要一个本地的注册地址和一名本地秘书，负责处理公司的法律和行政事务。"
    },
    {
      "story_name": "注册资本",
      "desc": "一个硬币的图标，旁边写着1新币，背景是淡黄色。",
      "cap": "注册资本最低只需1新币，没有上限，而且无需实际到位，是不是很灵活？"
    },
    {
      "story_name": "注册步骤1",
      "desc": "一个简单的流程图，第一步是命名公司，旁边有一个铅笔和纸张的图标。",
      "cap": "接下来，我们来看看注册的具体步骤。首先，确定公司名称，确保它符合新加坡会计与企业管理局的规定，不能与已有公司重名。"
    },
    {
      "story_name": "注册步骤2",
      "desc": "三个人物的剪影，分别代表股东、董事和秘书，旁边有一个文件夹的图标。",
      "cap": "然后，确定公司的股东、董事和秘书，准备好公司章程、注册申请表和注册地址证明等文件。"
    },
    {
      "story_name": "注册步骤3",
      "desc": "一个电脑屏幕的图标，显示在线申请的界面，旁边有一个支付符号。",
      "cap": "接着，通过ACRA的在线系统BizFile+递交注册申请，支付相应费用。"
    },
    {
      "story_name": "注册成功",
      "desc": "一张证书的图标，旁边有一个银行大楼的剪影。",
      "cap": "一旦注册成功，ACRA会颁发公司注册证书，你就可以为公司开立银行账户，开始日常的财务操作了。"
    },
    {
      "story_name": "注意事项1",
      "desc": "一个警告符号，旁边写着1000新币，背景是淡红色。",
      "cap": "当然，注册过程中也有一些需要注意的事项。比如，如果注册资本高于1000新币，后期公司转让时需要实缴等额的资金。"
    },
    {
      "story_name": "注意事项2",
      "desc": "护照、地址证明和银行证明信的图标，排列在一起。",
      "cap": "此外，非居民股东和董事需要提供额外的文件，如护照、地址证明和银行证明信。这些细节都需要提前准备好，以免耽误注册进程。"
    },
    {
      "story_name": "总结优势",
      "desc": "一个上升的箭头，旁边有地球和货币符号，背景是淡绿色。",
      "cap": "总的来说，注册新加坡公司不仅流程简单，而且优势明显。无论是提升企业形象，还是拓展国际市场，新加坡都能为你提供强大的支持。"
    },
    {
      "story_name": "咨询服务",
      "desc": "一个电话和对话框的图标，旁边有一个团队的剪影。",
      "cap": "如果你对新加坡公司注册、企业年审、挂名董事、银行开户、财税申报等流程有疑问，欢迎随时咨询我们。我们拥有丰富的经验和专业的团队，能够为你提供一站式服务，助你顺利在新加坡开展业务。"
    },
    {
      "story_name": "结束语",
      "desc": "点赞、关注和分享的社交媒体图标，排列在一起。",
      "cap": "最后，如果你觉得这些信息对你有帮助，别忘了点赞、关注，并分享给更多有需要的朋友。我们下期再见！"
    }
]

text_cap = [
    "你知道吗？",
    "注册新加坡公司",
    "可能比你想象中简单得多！",
    "而且",
    "它的优势远远超乎你的预期",
    "今天",
    "我就带你一探究竟",
    "看看为什么越来越多的企业家选择在新加坡注册公司",
    "首先",
    "新加坡作为亚洲金融中心",
    "拥有发达的金融市场和多元化的金融服务",
    "无论是想拓展国际市场",
    "还是规划税务",
    "新加坡都是一个理想的选择",
    "更重要的是",
    "新加坡实行低税率政策",
    "企业所得税率相对较低",
    "尤其是新小型企业",
    "还能享受税收减免的优惠",
    "再加上新加坡与许多国家签订了双重征税协定",
    "避免了双重征税的烦恼",
    "真是省心又省钱！",
    "那么",
    "注册新加坡公司到底需要哪些条件呢？",
    "其实并不复杂",
    "只要年满18周岁",
    "不限国籍",
    "都可以在新加坡注册公司",
    "不过",
    "董事中至少有一名必须是新加坡本地居民",
    "此外",
    "公司还需要一个本地的注册地址和一名本地秘书",
    "负责处理公司的法律和行政事务",
    "注册资本最低只需1新币",
    "没有上限",
    "而且无需实际到位",
    "是不是很灵活？",
    "接下来",
    "我们来看看注册的具体步骤",
    "首先",
    "确定公司名称",
    "确保它符合新加坡会计与企业管理局的规定",
    "不能与已有公司重名",
    "然后",
    "确定公司的股东、董事和秘书",
    "准备好公司章程、注册申请表和注册地址证明等文件",
    "接着",
    "通过ACRA的在线系统BizFile+递交注册申请",
    "支付相应费用",
    "一旦注册成功",
    "ACRA会颁发公司注册证书",
    "你就可以为公司开立银行账户",
    "开始日常的财务操作了",
    "当然",
    "注册过程中也有一些需要注意的事项",
    "比如",
    "如果注册资本高于1000新币",
    "后期公司转让时需要实缴等额的资金",
    "此外",
    "非居民股东和董事需要提供额外的文件",
    "如护照、地址证明和银行证明信",
    "这些细节都需要提前准备好",
    "以免耽误注册进程",
    "总的来说",
    "注册新加坡公司不仅流程简单",
    "而且优势明显",
    "无论是提升企业形象",
    "还是拓展国际市场",
    "新加坡都能为你提供强大的支持",
    "如果你对新加坡公司注册、企业年审、挂名董事、银行开户、财税申报等流程有疑问",
    "欢迎随时咨询我们",
    "我们拥有丰富的经验和专业的团队",
    "能够为你提供一站式服务",
    "助你顺利在新加坡开展业务",
    "最后",
    "如果你觉得这些信息对你有帮助",
    "别忘了点赞、关注",
    "并分享给更多有需要的朋友",
    "我们下期再见！"
  ]
timelines = [
    {
      "end": 888000,
      "start": 0
    },
    {
      "end": 2640000,
      "start": 888000
    },
    {
      "end": 4896000,
      "start": 2640000
    },
    {
      "end": 5832000,
      "start": 4896000
    },
    {
      "end": 8280000,
      "start": 5832000
    },
    {
      "end": 9024000,
      "start": 8280000
    },
    {
      "end": 10824000,
      "start": 9024000
    },
    {
      "end": 15288000,
      "start": 10824000
    },
    {
      "end": 16080000,
      "start": 15288000
    },
    {
      "end": 18576000,
      "start": 16080000
    },
    {
      "end": 22080000,
      "start": 18576000
    },
    {
      "end": 24360000,
      "start": 22080000
    },
    {
      "end": 25848000,
      "start": 24360000
    },
    {
      "end": 28320000,
      "start": 25848000
    },
    {
      "end": 29520000,
      "start": 28320000
    },
    {
      "end": 31776000,
      "start": 29520000
    },
    {
      "end": 34104000,
      "start": 31776000
    },
    {
      "end": 35952000,
      "start": 34104000
    },
    {
      "end": 38256000,
      "start": 35952000
    },
    {
      "end": 42720000,
      "start": 38256000
    },
    {
      "end": 44832000,
      "start": 42720000
    },
    {
      "end": 46680000,
      "start": 44832000
    },
    {
      "end": 47328000,
      "start": 46680000
    },
    {
      "end": 50664000,
      "start": 47328000
    },
    {
      "end": 52272000,
      "start": 50664000
    },
    {
      "end": 54048000,
      "start": 52272000
    },
    {
      "end": 55320000,
      "start": 54048000
    },
    {
      "end": 57672000,
      "start": 55320000
    },
    {
      "end": 58344000,
      "start": 57672000
    },
    {
      "end": 62400000,
      "start": 58344000
    },
    {
      "end": 63120000,
      "start": 62400000
    },
    {
      "end": 67176000,
      "start": 63120000
    },
    {
      "end": 70152000,
      "start": 67176000
    },
    {
      "end": 72672000,
      "start": 70152000
    },
    {
      "end": 73800000,
      "start": 72672000
    },
    {
      "end": 75648000,
      "start": 73800000
    },
    {
      "end": 77064000,
      "start": 75648000
    },
    {
      "end": 78024000,
      "start": 77064000
    },
    {
      "end": 80304000,
      "start": 78024000
    },
    {
      "end": 81096000,
      "start": 80304000
    },
    {
      "end": 82560000,
      "start": 81096000
    },
    {
      "end": 86304000,
      "start": 82560000
    },
    {
      "end": 88224000,
      "start": 86304000
    },
    {
      "end": 89064000,
      "start": 88224000
    },
    {
      "end": 91872000,
      "start": 89064000
    },
    {
      "end": 96840000,
      "start": 91872000
    },
    {
      "end": 97584000,
      "start": 96840000
    },
    {
      "end": 101760000,
      "start": 97584000
    },
    {
      "end": 103224000,
      "start": 101760000
    },
    {
      "end": 104712000,
      "start": 103224000
    },
    {
      "end": 107208000,
      "start": 104712000
    },
    {
      "end": 109776000,
      "start": 107208000
    },
    {
      "end": 111912000,
      "start": 109776000
    },
    {
      "end": 112656000,
      "start": 111912000
    },
    {
      "end": 115560000,
      "start": 112656000
    },
    {
      "end": 116280000,
      "start": 115560000
    },
    {
      "end": 118800000,
      "start": 116280000
    },
    {
      "end": 122112000,
      "start": 118800000
    },
    {
      "end": 122832000,
      "start": 122112000
    },
    {
      "end": 126312000,
      "start": 122832000
    },
    {
      "end": 129456000,
      "start": 126312000
    },
    {
      "end": 132096000,
      "start": 129456000
    },
    {
      "end": 133920000,
      "start": 132096000
    },
    {
      "end": 135000000,
      "start": 133920000
    },
    {
      "end": 137952000,
      "start": 135000000
    },
    {
      "end": 139728000,
      "start": 137952000
    },
    {
      "end": 141744000,
      "start": 139728000
    },
    {
      "end": 143664000,
      "start": 141744000
    },
    {
      "end": 146592000,
      "start": 143664000
    },
    {
      "end": 154224000,
      "start": 146592000
    },
    {
      "end": 155976000,
      "start": 154224000
    },
    {
      "end": 158736000,
      "start": 155976000
    },
    {
      "end": 160896000,
      "start": 158736000
    },
    {
      "end": 163464000,
      "start": 160896000
    },
    {
      "end": 164304000,
      "start": 163464000
    },
    {
      "end": 166896000,
      "start": 164304000
    },
    {
      "end": 168816000,
      "start": 166896000
    },
    {
      "end": 171120000,
      "start": 168816000
    },
    {
      "end": 172416000,
      "start": 171120000
    }
  ]
pic_timelines = []

In [19]:
l,L = len(input_list),len(text_cap)
start,end,idx = 0,0,0
for i in range(l):
    while idx < L:
        if text_cap[idx] in input_list[i]['cap']:
            end = timelines[idx]['end']
            idx += 1
        else:
            break

    pic_timelines.append({
        "start":start,
        "end":end
    })
    start = end
pic_timelines

[{'start': 0, 'end': 15288000},
 {'start': 15288000, 'end': 28320000},
 {'start': 28320000, 'end': 38256000},
 {'start': 38256000, 'end': 46680000},
 {'start': 46680000, 'end': 70152000},
 {'start': 70152000, 'end': 77064000},
 {'start': 77064000, 'end': 88224000},
 {'start': 88224000, 'end': 96840000},
 {'start': 96840000, 'end': 103224000},
 {'start': 103224000, 'end': 111912000},
 {'start': 111912000, 'end': 122112000},
 {'start': 122112000, 'end': 133920000},
 {'start': 133920000, 'end': 146592000},
 {'start': 146592000, 'end': 163464000},
 {'start': 163464000, 'end': 172416000}]

In [1]:
import tempfile
tempfile.gettempdir()

'C:\\Users\\EDY\\AppData\\Local\\Temp'